## ETL
importamos librerias

In [3]:
import pandas as pd
import json


### Games dataset
carga, transformacion y guardado en csv de la tabla games

In [5]:
# Cargamos el json en un dataframe
data = []
with open('output_steam_games.json') as e:
    for line in e:
        data.append(json.loads(line))
dfgames = pd.DataFrame(data)

In [6]:
dfgames = dfgames.dropna() #drop valores faltantes

In [7]:
dfgames.columns #chequeo que columnas voy a eliminar

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [8]:
#drop de columnas que no necesito
dfgames = dfgames.drop(['publisher', 'reviews_url','url','tags', 'app_name', 'specs', 'price', 'early_access', 'developer'], axis=1)

Genero dummies para los generos, lo cual me permitira categorizar de manera mas facil

In [38]:
# Creamos dummies para genres
dfgames['genres'] = dfgames['genres'].fillna('[]')  # Rellenar los valores faltantes con una lista vacía
dfgames['genres'] = dfgames['genres'].apply(lambda x: ', '.join(x))  # Convertir la lista de géneros a una cadena separada por comas

# Crear variables ficticias para los géneros
dummy_genres = dfgames['genres'].str.get_dummies(', ')

# Concatenar las variables ficticias con el DataFrame original
dfgames_with_dummies = pd.concat([dfgames, dummy_genres], axis=1)

                                                 genres  \
88310       Action, Casual, Indie, Simulation, Strategy   
88311                Free to Play, Indie, RPG, Strategy   
88312   Casual, Free to Play, Indie, Simulation, Sports   
88313                         Action, Adventure, Casual   
88315                     Action, Adventure, Simulation   
...                                                 ...   
120439                 Action, Adventure, Casual, Indie   
120440              Casual, Indie, Simulation, Strategy   
120441                          Casual, Indie, Strategy   
120442                        Indie, Racing, Simulation   
120443                                    Casual, Indie   

                           title release_date      id  Action  Adventure  \
88310        Lost Summoner Kitty   2018-01-04  761140       1          0   
88311                  Ironbound   2018-01-04  643980       0          0   
88312    Real Pool 3D - Poolians   2017-07-24  670290       0  

In [39]:
dfgames = dfgames_with_dummies

Cambio la columna 'release_date' a year. Para trabajar unicamente el año en lugar de un datetime

In [40]:
# Reemplazar los valores no válidos por NaN
dfgames["release_date"] = pd.to_datetime(dfgames["release_date"], errors='coerce')
# Filtrar el DataFrame para obtener los registros válidos
df_filtered = dfgames.dropna(subset=["release_date"])
# Convertir la columna "release_date" a tipo datetime
df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])
# Rellenando los valores faltantes en las columna'release_date' con la moda
columns_fill_mode = ['release_date']
for col in columns_fill_mode:
    dfgames[col].fillna(dfgames[col].mode()[0], inplace=True)
# Modelado y eliminación de la columna 'release_date'
dfgames['year'] = dfgames['release_date'].dt.year.astype('Int64')
dfgames.drop('release_date', axis=1, inplace=True)

C:\Users\W10-PC\AppData\Local\Temp\ipykernel_12596\339929420.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["release_date"] = pd.to_datetime(df_filtered["release_date"])


In [41]:
dfgames

,genres,title,id,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,year
88310,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,761140,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,2018
88311,"Free to Play, Indie, RPG, Strategy",Ironbound,643980,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,2018
88312,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,670290,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,2017
88313,"Action, Adventure, Casual",弹炸人2222,767400,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2017
88315,"Action, Adventure, Simulation",Battle Royale Trainer,772540,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,"Action, Adventure, Casual, Indie",Kebab it Up!,745400,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2018
120440,"Casual, Indie, Simulation, Strategy",Colony On Mars,773640,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,2018
120441,"Casual, Indie, Strategy",LOGistICAL: South Africa,733530,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,2018
120442,"Indie, Racing, Simulation",Russian Roads,610660,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,2018


In [27]:
dfgames.columns

Index(['genres', 'title', 'id', 'Action', 'Adventure',
       'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing', 'year'],
      dtype='object')

In [26]:
dfgames.to_csv('GamesClean.csv') #Lo guardo en un csv para trabajarlo mas facil

### Reviews dataset
Cargado, transformacion y guardado de la tabla reviews en csv

In [58]:
with open("australian_user_reviews.json", 'r', encoding='utf-8') as w:
    data = w.readlines()
    
datos = [eval(line.strip()) for line in data]

df_review = pd.DataFrame(datos)
df_review.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Desanidado

In [59]:
df_reviewsOpen = df_review.explode('reviews') #creo un df con el archivo review y accedo ala column que me interesa desanidar.
dfreviewsOpen = pd.concat([df_reviewsOpen.drop(['reviews'], axis=1), df_reviewsOpen['reviews'].apply(pd.Series)], axis=1) #con esta expresion  creo un nuevo df donde esta la info desanidada en columnas

In [60]:
# se cre la columna year
dfreviewsOpen['posted year'] = dfreviewsOpen['posted'].str.extract(r'(\d{4})')

#Se elimina la columna posted
dfreviewsOpen.drop('posted' , axis = 1, inplace = True)

In [61]:
dfreviewsOpen.head(3)

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,review,0,posted year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN,2011


In [11]:
print(type(dfreviewsOpen['review']))

<class 'pandas.core.series.Series'>


### Implementacion del Sentiment Score

In [62]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to assign values according to the scale
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Return neutral if it is empty or NaN
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 0  # Good score
        elif compound_score <= -0.05:
            return 2  # Bad score
        else:
            return 1
    else:
        return 1  # Return neutral for non-string values


# Convertir la columna 'review' a tipo de dato str
dfreviewsOpen['review'] = dfreviewsOpen['review'].astype(str)

# Aplicar la función get_sentiment_score a la columna 'review'
dfreviewsOpen['sentiment_score'] = dfreviewsOpen['review'].apply(get_sentiment_score)

# Imprimir el DataFrame actualizado
print(dfreviewsOpen)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\W10-PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                 user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1                js41637               http://steamcommunity.com/id/js41637   
1                js41637               http://steamcommunity.com/id/js41637   
...                  ...                                                ...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                                  funny last_edited

In [63]:
dfreviewsOpen.to_csv('ReviewsClean.csv') #nuevamente en un csv

### Items Dataset
Carga, transformacion y guardado en csv de la tabla items

In [42]:
with open("australian_users_items.json", 'r', encoding='utf-8') as i:
    data = i.readlines()
    
records = [eval(line.strip()) for line in data]

df_item = pd.DataFrame(records)
df_item.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [21]:
#Verifico el contenido de items para ver que quiero sacar
print(df_item['items'][0])

[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 7, 'playtime_2weeks': 0}, {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '60', 'item_name': 'Ricochet', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '70', 'item_name': 'Half-Life', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_forever': 4733, 'playtime_2weeks': 0}, {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_forever': 1853, 'playtime_2weeks': 0}, {'item_i

Desanidado

In [44]:
# aplico la funcion lambda para desanidar las keys que busco con sus values
df_item['playtime_forever'] = df_item['items'].apply(lambda x: x[0].get('playtime_forever') if len(x) > 0 else 0)
df_item['playtime_2weeks'] = df_item['items'].apply(lambda x: x[0].get('playtime_2weeks') if len(x) > 0 else 0)
df_item['id'] = df_item['items'].apply(lambda x: x[0].get('item_id') if len(x) > 0 else 0)
df_item['playtime_forever'] = df_item['playtime_forever'].astype(int)
df_item['playtime_2weeks'] = df_item['playtime_2weeks'].astype(int)


In [45]:
df_item.drop(['user_url'], axis=1, inplace=True)  # dropeamos columnas que no usaremos
df_item.drop(['items'], axis=1, inplace=True)

In [48]:
df_item #chequeo como quedaron

,user_id,items_count,steam_id,playtime_forever,playtime_2weeks,id
0,76561197970982479,277,76561197970982479,6,0,10
1,js41637,888,76561198035864385,0,0,10
2,evcentric,137,76561198007712555,923,0,1200
3,Riot-Punch,328,76561197963445855,0,0,10
4,doctr,541,76561198002099482,1131,0,300
...,...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,0,0,413850
88306,76561198326700687,177,76561198326700687,0,0,11020
88307,XxLaughingJackClown77xX,0,76561198328759259,0,0,0
88308,76561198329548331,7,76561198329548331,677,677,304930


In [ ]:
df_item.to_csv('ItemsClean.csv', index=False) #guardado en csv

### Creacion del dataset final

In [ ]:
# A travez de id podemos unir tablas para ahorrar espacio
merged_df = pd.merge(dfgames, df_item, on='id') # Convino las tablas previamente desanidadas

In [56]:
merged_df.drop(['genres'],axis=1, inplace=True) #no necesitamos genres ya

In [58]:
merged_df.shape #chequeo las dimensiones nuevas

(62229, 29)

In [59]:
merged_df.to_csv('Df_Final.csv', index=False) # guardo en csv

In [37]:
merged_df.columns #chequeo columnas

Index(['title', 'id', 'Action', 'Adventure', 'Animation &amp; Modeling',
       'Audio Production', 'Casual', 'Design &amp; Illustration',
       'Early Access', 'Education', 'Free to Play', 'Indie',
       'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation',
       'Software Training', 'Sports', 'Strategy', 'Utilities',
       'Video Production', 'Web Publishing', 'year', 'user_id', 'items_count',
       'steam_id', 'playtime_forever', 'playtime_2weeks'],
      dtype='object')

In [65]:
df_review = pd.read_csv('ReviewsClean.csv') #cargamos reviews para unirlo posteriormente

In [66]:
df_final = pd.merge(merged_df, df_review, on = 'user_id') #unimos el dataset convinado con el dataset Items

In [67]:
df_final.to_csv('df_final.csv', index=False) #guardamos dataset final

In [68]:
df_final.columns #ultima verificacion de columnas para borrar

Index(['title', 'id', 'Action', 'Adventure', 'Animation &amp; Modeling',
       'Audio Production', 'Casual', 'Design &amp; Illustration',
       'Early Access', 'Education', 'Free to Play', 'Indie',
       'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation',
       'Software Training', 'Sports', 'Strategy', 'Utilities',
       'Video Production', 'Web Publishing', 'year', 'user_id', 'items_count',
       'steam_id', 'playtime_forever', 'playtime_2weeks', 'Unnamed: 0',
       'user_url', 'funny', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', '0', 'posted year', 'sentiment_score'],
      dtype='object')

In [69]:
df_final.drop(columns=['0', 'Unnamed: 0'], inplace=True) #borramos columnas agregadas del merge

In [70]:
df_final.to_csv('df_final.csv', index=False) #Dataset Final